In [ ]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions

In [ ]:
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)


In [ ]:
driver.get('https://www.twitter.com/login')

In [ ]:
username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('ptrabi')

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys('siMCp7tfvBw8YL')

In [ ]:
password.send_keys(Keys.RETURN)

In [ ]:
search_input = driver.find_element_by_xpath('//input[@enterkeyhint="search"]')
search_input.send_keys('#gardening')


In [ ]:
search_input.send_keys(Keys.RETURN)

In [ ]:
driver.find_element_by_link_text('Latest').click()

In [ ]:
cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')

In [ ]:
card = cards[2]

In [ ]:
#username
card.find_element_by_xpath('.//span').text

In [ ]:
#twitter handle
card.find_element_by_xpath('.//span[contains(text(),"@")]').text

In [ ]:
#time
card.find_element_by_xpath('.//time').get_attribute('datetime')

In [ ]:
#time
comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
comment+responding

In [ ]:
#retweet
retweet = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
print("Retweet "+ retweet)
#likes
likes = card.find_element_by_xpath('.//div[@data-testid="like"]').text
print("likes "+likes)

In [ ]:
def get_tweet_data(card):
    name = card.find_element_by_xpath('.//span').text
    username = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    try: 
        postDate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text=comment+responding
    replyCount = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweetCount = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    likeCount = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (name, username, postDate, text, replyCount, retweetCount, likeCount)
    return tweet

In [ ]:
get_tweet_data(card)

In [ ]:
tweet_data = []

for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)
        

In [ ]:
tweet_data[0]

In [ ]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

# Combine all

In [ ]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions

def get_tweet_data(card):
    name = card.find_element_by_xpath('.//span').text
    username = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    try: 
        postDate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text=comment+responding
    replyCount = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweetCount = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    likeCount = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (name, username, postDate, text, replyCount, retweetCount, likeCount)
    return tweet

#buka browser
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options) 


#login
driver.get('https://www.twitter.com/login')
driver.maximize_window

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('ptrabi')

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys('J@jd9SL9qxth*J*Z!uuXz#56uXCi')

password.send_keys(Keys.RETURN)
sleep(1)

#search

search_input = driver.find_element_by_xpath('//input[@enterkeyhint="search"]')
search_input.send_keys('#gardening')
search_input.send_keys(Keys.RETURN)
sleep(2)
#latest tab
driver.find_element_by_link_text('Latest').click()
sleep(2)
#get 15 latest tweets
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break
        
driver.close() 

In [ ]:
len(data)

In [ ]:
with open('turkcell_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Emojis', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)